# DGL 2026 Coursework 1

- Explanations should be supported by evidence (e.g. plots, statistics, or controlled comparisons), not only intuition.
- Plots or metrics must be accompanied by an explanation of what they reveal and why this explains the model behavior
- This coursework is completed in a **single Jupyter notebook**.  
- You may reuse helper functions across questions unless explicitly stated otherwise.  
- Your final submission **must run correctly with “Restart Kernel & Run All”**.




## Question 1 - Node Classification in Heterogeneous Spaces

You are given a **bipartite** graph with two node types:
- **Drugs**: chemical fingerprints (sparse vectors)
- **Proteins**: sequence embeddings (dense vectors)

The feature spaces are **disjoint** (different semantics, different dimensions). You must predict **drug labels**.

### Rules
- ✅ Allowed: Python, NumPy, PyTorch, matplotlib, scikit-learn, seaborn
- ❌ Not allowed: DGL, PyG, or any prebuilt GNN/attention layers




In [1]:
!pip install --upgrade pip
!pip install -r requirements.txt    

In [2]:
# =========================
# Standard Library
# =========================
import os
import json
import math
import time
import random

# =========================
# Numerical & Scientific
# =========================
import numpy as np

# =========================
# PyTorch
# =========================
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# =========================
# Graph & Geometry
# =========================
import networkx as nx
from graph_ricci_curvature import GraphRicciCurvature

# =========================
# Visualisation
# =========================
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE

# =========================
# Evaluation & Analysis
# =========================
from sklearn.metrics import precision_recall_fscore_support

# =========================
# Reproducibility
# =========================
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(42)

# =========================
# Device
# =========================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")



Using device: cpu


### 1.1 Dataset (10 pts)

#### Load dataset
The dataset is stored as `.npz` and contains:
- `Xd`: drug features `[n_drugs, d_drug]`
- `Xp`: protein features `[n_proteins, d_protein]`
- `y`: drug labels `[n_drugs]`
- `edges_dp`: edges `[E,2]` of `(drug_id, protein_id)`
- `train_mask`, `val_mask`, `test_mask`: boolean masks over drugs


In [ ]:
DATA_PATH = "data/synth_drug_protein_np.npz"  # update path if needed
assert os.path.exists(DATA_PATH), f"Dataset not found: {DATA_PATH}"

data = np.load(DATA_PATH, allow_pickle=True)

Xd = torch.tensor(data["Xd"], dtype=torch.float32)
Xp = torch.tensor(data["Xp"], dtype=torch.float32)
y  = torch.tensor(data["y"], dtype=torch.long)
edges_dp = torch.tensor(data["edges_dp"], dtype=torch.long)  # [E,2]

train_mask = torch.tensor(data["train_mask"], dtype=torch.bool)
val_mask   = torch.tensor(data["val_mask"], dtype=torch.bool)
test_mask  = torch.tensor(data["test_mask"], dtype=torch.bool)

n_drugs, d_drug = Xd.shape
n_proteins, d_protein = Xp.shape
E = edges_dp.shape[0]
n_classes = int(y.max().item()) + 1

print("Xd:", Xd.shape)
print("Xp:", Xp.shape)
print("edges_dp:", edges_dp.shape)
print("y:", y.shape, "classes:", n_classes)
print("splits:", train_mask.sum().item(), val_mask.sum().item(), test_mask.sum().item())

#### 1.1.a Exploratory Data Analysis (2 pts)
**Task:** Calculate the following statistics to understand the "Semantic Gap."
1. The **sparsity** (percentage of zeros) of the Drug features.
2. The **mean and standard deviation** of the Protein features.
3. The **average degree** (number of protein neighbors) per drug.

In [ ]:
def analyze_feature_gap(Xd, Xp, edges_dp):
    # TODO: Calculate and print the requested statistics
    pass

analyze_feature_gap(Xd, Xp, edges_dp)

#### 1.1.b Feature Density Analysis (2 pts)
A critical part of working with heterogeneous graphs is understanding the numerical distribution of different node types. In this bipartite graph, Drugs and Proteins come from entirely different semantic spaces (binary fingerprints vs. dense embeddings).

Task: Implement the `plot_feature_density` function to visualize the distribution of average feature values for all Drugs versus all Proteins.

In [ ]:
def plot_feature_density(Xd, Xp):
    """
    Plots the density of average feature values for all Drugs vs all Proteins.

    Args:
        Xd: Drug features [n_drugs, d_drug]
        Xp: Protein features [n_proteins, d_protein]
    """
    plt.figure(figsize=(10, 6))

    # 1. Calculate average feature values per node (mean across the feature dimension)
    # TODO: Calculate avg_d and avg_p

    # 2. Use sns.kdeplot to visualize the distributions
    # Hint: Use fill=True and distinct colors to show the two "islands" of data.

    # =========================
    # Your code starts here
    # =========================

    # =========================
    # Your code ends here
    # =========================

    plt.title("Distribution of *average* feature values: Drugs vs Proteins")
    plt.xlabel("Average feature value")
    plt.ylabel("Density")
    plt.legend(["Drugs", "Proteins"])
    plt.show()

# plot_feature_density(Xd, Xp)

#### 1.1.c t-SNE Gap Visualization (2 pts)
**Task:** Use t-SNE to project both drugs and proteins into a 2D space.

**Hint:** Drug and protein features have different dimensions and cannot be concatenated directly.
You must first project them into the same dimension using a fixed random projection
(e.g., 128 dimensions) before running t-SNE.


In [ ]:
set_seed(42)

def fixed_random_projection(X, d_out=128, seed=0):
    """
    Projects features to a shared dimension using a fixed random matrix.
    This projection is non-learnable and used only for visualization.
    """
    g = torch.Generator(device=X.device)
    g.manual_seed(seed)
    R = torch.randn(X.size(1), d_out, generator=g, device=X.device)
    R = R / (X.size(1) ** 0.5)
    return X @ R


In [ ]:
set_seed(42)

def plot_feature_spaces(Xd, Xp, n_each=400):
    # TODO:
    # 1. Subsample nodes for fast execution
    # 2. Project Xd and Xp to a shared 128-dim space using a fixed random matrix
    # 3. Run TSNE (from sklearn, check imports) and plot with different colors for drugs/proteins
    pass

plot_feature_spaces(Xd, Xp)

#### 1.1.d Graph Visualization (2 pts)
You must visualize **a small part** of the bipartite graph:
- sample `n_drug_sample` drugs
- include their connected proteins
- draw the bipartite subgraph

Your plot should make the bipartite structure obvious (two vertical columns: drugs on left, proteins on right).

In [ ]:
def sample_bipartite_subgraph(edges_dp: torch.Tensor,
                              n_drugs: int,
                              n_drug_sample: int = 30,
                              seed: int = 42):
    """
    Sample a subgraph:
      - choose n_drug_sample drugs uniformly at random
      - include all proteins they connect to
      - return lists of (drug_ids, protein_ids, edges_sub)

    edges_dp: [E,2] (drug_id, protein_id), does NOT need to be sorted for this function
    """
    # =========================
    # Your code starts here
    # =========================
    # TODO:
    # 1) sample drug ids
    # 2) filter edges where drug in sampled set
    # 3) collect unique protein ids appearing in those edges
    # 4) reindex node ids locally for plotting convenience (optional but recommended)
    raise NotImplementedError
    # =========================
    # Your code ends here
    # =========================


def plot_bipartite_subgraph(drug_ids_global,
                            prot_ids_global,
                            edges_sub_global,
                            title: str = "Bipartite subgraph (Drugs ↔ Proteins)",
                            max_edges: int = 300):
    """
    Plot bipartite graph with a simple two-column layout:
      - drugs at x=0
      - proteins at x=1
    and edges drawn as line segments.

    If there are too many edges, you may subsample edges to max_edges for readability.
    """
    # =========================
    # Your code starts here
    # =========================
    # TODO:
    # 1) assign y-positions to drugs and proteins (e.g., equally spaced)
    # 2) draw nodes as scatter points (different marker/color for drugs vs proteins)
    # 3) draw edges as faint lines
    # 4) add legend/title
    raise NotImplementedError
    # =========================
    # Your code ends here
    # =========================


# Example usage (uncomment after implementation)
# drug_ids_g, prot_ids_g, edges_sub_g = sample_bipartite_subgraph(edges_dp, n_drugs=n_drugs, n_drug_sample=30)
# plot_bipartite_subgraph(drug_ids_g, prot_ids_g, edges_sub_g)


#### 1.1.e Theoretical analysis (2 pts)
Consider a naive approach: project drugs and proteins into the same hidden dimension with linear layers, then mean/sum aggregate.

Explain why this is suboptimal in this bipartite setting, referring to:
- **Over-smoothing / signal dilution**
- **Manifold alignment**

Write your answer below.

### 1.2 Cross-Modal Attention (25 pts)

In this section, you will implement a custom Cross-Modal Attention (CMA) layer to update drug node representations using information from their protein neighbors. Because the graph is bipartite and the feature spaces are disjoint, we treat the **Drug** as the Query and the **Proteins** as Keys and Values.

#### Mathematical Formulation

The updated representation for a drug node $u$ is calculated using a residual connection:

$$h_u^{(l+1)} = \sigma\left(\sum_{v \in N(u)} \alpha_{uv} W_{val} h_v^{(l)} + h_u^{(l)}\right)$$

The attention coefficient $\alpha_{uv}$ weights the importance of protein $v$ to drug $u$. It is computed via a neighborhood-specific softmax:

$$\alpha_{uv} = \text{softmax}_{v \in N(u)} \left( \text{LeakyReLU} \left( a^T [W_q h_u^{(l)} \Vert W_k h_v^{(l)}] \right) \right)$$



#### Symbol Definitions

* $h_u^{(l)}$: The feature vector of the **central drug** node $u$ at layer $l$.
* $h_v^{(l)}$: The feature vector of the **neighboring protein** node $v$ at layer $l$.
* $N(u)$: The set of protein neighbors connected to drug $u$.
* $W_q$: Learnable weight matrix for the **Query** transformation (applied to Drugs).
* $W_k$: Learnable weight matrix for the **Key** transformation (applied to Proteins).
* $W_{val}$: Learnable weight matrix for the **Value** transformation (applied to Proteins).
* $a^T$: A learnable attention vector that reduces the concatenated Query-Key vector to a single scalar score.
* $\Vert$: The concatenation operation.
* $\sigma$: A non-linear activation function (e.g., ReLU).

#### Requirements

* **No Pre-built Layers**: You must implement the logic using basic PyTorch operations. You may use `nn.Linear` or `nn.Parameter` to define the weights.
* **Query/Key Separation**: Ensure $W_q$ is only used for drug nodes and $W_k, W_{val}$ are only used for protein nodes.
* **Neighborhood Softmax**: The softmax must be normalized strictly over the neighbors of each individual drug. Use the `ptr_drug` array to identify these segments.

#### 1.2.a Segment softmax helper (5 pts)
In a standard GNN, we often have a long 1D tensor of scores (one for every edge in the graph). However, attention must be a probability distribution **per node neighborhood**. To achieve this, we use a "segment softmax."

#### How it works
Since `edges_dp` is sorted by the drug (query) node index, all proteins connected to Drug 0 appear first, followed by all proteins for Drug 1, and so on. The `ptr_drug` array acts as a map to these segments:
- The neighbors of drug $u$ are found in the slice `scores[ptr[u] : ptr[u+1]]`.
- You must apply the softmax operation independently to each of these slices.

#### Numerical Stability
When implementing softmax, it is a best practice to use the "Log-Sum-Exp" trick or subtract the maximum value from the scores before exponentiating:
$$\text{softmax}(x_i) = \frac{\exp(x_i - \max(x))}{\sum \exp(x_j - \max(x))}$$
This prevents `torch.exp()` from overflowing if the attention scores are large.



#### Implementation Task
Implement the function below. While high-performance GNNs use specialized kernels, for this coursework, a Python loop over the number of drugs is acceptable.

In [ ]:
def segment_softmax(scores: torch.Tensor, ptr: torch.Tensor) -> torch.Tensor:
    """

    Computes a numerically stable softmax over segments of an edge tensor.

    Each segment corresponds to the neighborhood of a single drug node. This
    ensures that the attention weights for each drug's protein neighbors
    sum to 1.0.

    Args:
        scores: Tensor of shape [E] (one scalar per edge)
        ptr: Tensor of shape [num_nodes + 1] defining neighborhood segments
    Returns:
        Tensor of shape [E], where softmax is applied independently per segment.

    Notes:
        To ensure numerical stability and avoid overflow, you should
        subtract the maximum score within each segment before exponentiating:
        softmax(x_i) = exp(x_i - max(x)) / sum(exp(x_j - max(x)))
    """
    # =========================
    # Your code starts here
    # =========================
    # TODO: Implement the stable per-segment softmax using a loop over drugs.
    # 1. Identify segment bounds using ptr.
    # 2. skip empty segments (if any drug has no neighbors)
    # 3. Compute max per segment for stability.
    # 4. Exponentiate and normalize.
    raise NotImplementedError
    # =========================
    # Your code ends here
    # =========================


#### 1.2.b Cross-Modal Attention Layer (5 pts)

Now you will implement the `CrossModalAttention` module. This layer is the core of the architecture, allowing drug nodes to "listen" to their protein neighbors and update their internal state based on biological relevance.

#### Functional Requirements:
1.  **Shared Latent Space**: Since drugs ($d_{drug}$) and proteins ($d_{protein}$) have different input dimensions, you must project them into a shared hidden dimension $d_{hidden}$ using learnable weights.
2.  **Directional Processing**: In this bipartite setup, the drugs act as **Queries** ($Q$), while the proteins act as **Keys** ($K$) and **Values** ($V$).
3.  **The Attention Mechanism**:
    * For every edge $(u, v)$, compute a scalar score based on the concatenation of the drug query $q_u$ and protein key $k_v$.
    * Apply a `LeakyReLU` activation to allow the model to learn non-linear relationships.
    * Use your `segment_softmax` to normalize these scores into weights $\alpha_{uv}$ that sum to 1 for each drug's neighborhood.
4.  **Message Aggregation**: Multiply the protein values $v_v$ by the attention weights $\alpha_{uv}$ and sum them up for each drug.
5.  **Residual Connection**: Add the aggregated neighbourhood information back to the projected drug representation ($d_{hidden}$) to preserve the drug's own feature identity.



#### Implementation Task
Implement the `forward` pass using only basic PyTorch operations. Avoid using `for` loops where possible for the projection and score calculation; however, a loop is acceptable for the final aggregation across segments.

In [ ]:
# =========================
# Your code starts here
# =========================
class CrossModalAttention(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__()

        raise NotImplementedError


    def forward(self, *args, **kwargs):
        """Return updated drug reps."""

        raise NotImplementedError
    
# =========================
# Your code ends here
# =========================


#### 1.2.c Discussion (10 pts)

**Since the task is drug classification but the graph is bipartite, why is a 1-layer model structurally incapable of exploiting drug–drug information for this task? Explain mathematically.**

**[Answer]**

...


#### 1.2.d Heterogeneous CMA Classifier (5 pts)
We recommend a **2-step** update to allow information flow via drug→protein→drug (2-hop paths).

You will implement:
- drug classifcation model using CMA layers
- protein update from drugs
- drug update from proteins

See the docstrings in the class below

In [ ]:
set_seed(42)  

# ---------------------------------------------------------------------
# Helper functions for edge re-indexing and CSR-style pointer creation
# Provided utility code (you do not need to modify).
# ---------------------------------------------------------------------

def sort_edges_by_src(edges: torch.Tensor) -> torch.Tensor:
    """
    Sort edges by their source node index.

    Parameters
    ----------
    edges : torch.Tensor, shape (E, 2)
        Edge list where each row is (src, dst).

    Returns
    -------
    torch.Tensor, shape (E, 2)
        Edge list sorted in ascending order of src.

    Notes
    -----
    Sorting edges by source node is required before constructing
    CSR-style pointer arrays (ptr), which assume contiguous blocks
    of edges per source node.
    """
    src = edges[:, 0]                 # Extract source node indices
    order = torch.argsort(src)        # Indices that sort edges by src
    return edges[order]               # Reorder edges accordingly


def make_ptr_from_sorted_src(sorted_src: torch.Tensor, n_src: int) -> torch.Tensor:
    """
    Construct a CSR-style pointer array from sorted source indices.

    Parameters
    ----------
    sorted_src : torch.Tensor, shape (E,)
        Source node indices sorted in ascending order.
    n_src : int
        Total number of source nodes.

    Returns
    -------
    torch.Tensor, shape (n_src + 1,)
        Pointer array where ptr[i] gives the start index of edges
        originating from source node i in the sorted edge list.

    Notes
    -----
    This follows the Compressed Sparse Row (CSR) convention:
    - Edges for node i are located in the range [ptr[i], ptr[i+1]).
    - Nodes with no outgoing edges will have ptr[i] == ptr[i+1].
    """
    counts = torch.bincount(sorted_src, minlength=n_src)  # Number of edges per source
    ptr = torch.zeros(n_src + 1, dtype=torch.long)        # Allocate pointer array
    ptr[1:] = torch.cumsum(counts, dim=0)                 # Cumulative sum of counts
    return ptr


def make_reverse_edges(edges_dp: torch.Tensor) -> torch.Tensor:
    """
    Reverse the direction of edges.

    Parameters
    ----------
    edges_dp : torch.Tensor, shape (E, 2)
        Edge list in (drug_id, protein_id) format.

    Returns
    -------
    torch.Tensor, shape (E, 2)
        Edge list in (protein_id, drug_id) format.

    Notes
    -----
    This is useful when switching the perspective of the graph,
    e.g. from drug-centric neighborhoods to protein-centric ones.
    """
    return torch.stack([edges_dp[:, 1], edges_dp[:, 0]], dim=1)


# ---------------------------------------------------------------------
# Build protein-centric edge representation
# ---------------------------------------------------------------------

edges_pd = sort_edges_by_src(make_reverse_edges(edges_dp))
# edges_pd now has shape (E, 2) with (protein_id, drug_id),
# sorted by protein_id

prot_src = edges_pd[:, 0]   # Protein node indices (sources)
drug_dst = edges_pd[:, 1]   # Drug node indices (destinations)

ptr_prot = make_ptr_from_sorted_src(prot_src, n_proteins)
# ptr_prot defines edge ranges for each protein node in edges_pd

print("edges_pd:", edges_pd.shape)
print("ptr_prot:", ptr_prot.shape)


In [ ]:
# =========================
# Your code starts here
# =========================
class HeteroCMAClassifier(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__()

        raise NotImplementedError

    def forward(self, *args, **kwargs):
        raise NotImplementedError
# =========================
# Your code ends here
# =========================


### 1.3 Alignment Loss (5 pts)

You must implement:
$$L_{total} = L_{CE} + \lambda L_{align}$$

Where `L_align` pulls **connected** drug-protein pairs closer in a **shared latent space**.

Hint: Consider distance measures between two embedding vectors.




#### 1.3.a Implementation (2 pts)
Implement `alignment_loss(...)` for connected edges only.
Define `z_d` and `z_p` as the **d_hidden-dimensional embeddings** produced by your model (after projection into the shared latent space and before the final classifier). Alignment loss should only be computed for connected drug-protein pairs.

In [ ]:
def alignment_loss(z_d: torch.Tensor, z_p: torch.Tensor, edges_dp: torch.Tensor) -> torch.Tensor:
    """
    z_d: drug embeddings in shared alignment space [n_drugs, d_align]
    z_p: protein embeddings in shared alignment space [n_proteins, d_align]
    edges_dp: [E,2]
    Return scalar alignment loss.
    """
    # =========================
    # Your code starts here
    # =========================
    # TODO: implement an alignment loss over connected pairs only
    raise NotImplementedError
    # =========================
    # Your code ends here
    # =========================


#### Training utilities
Implement:
- a macro-F1 function
- training loop that compares λ=0 vs λ>0 (only change λ, do not change other hyperparameters, number of layers etc.)
- testing loop

You must plot F1 curves and discuss the trade-off.


In [ ]:
def macro_f1(pred: torch.Tensor, target: torch.Tensor, n_classes: int) -> float:
    # =========================
    # Your code starts here
    # =========================
    # TODO: implement macro-F1
    raise NotImplementedError
    # =========================
    # Your code ends here
    # =========================


def train_model(*args, **kwargs):
    """Return a dict history with train/val F1 and losses."""
    # =========================
    # Your code starts here
    # =========================
    # TODO:
    # 1) move tensors to device
    # 2) define optimizer
    # 3) loop epochs:
    #    - forward
    #    - compute CE loss on train drugs
    #    - compute alignment loss if lam>0
    #    - backprop + step
    #    - evaluate macro-F1 on train and val
    # 4) return history
    raise NotImplementedError
    # =========================
    # Your code ends here
    # =========================

def test_model(*args, **kwargs):
    # =========================
    # Your code starts here
    # =========================
    raise NotImplementedError
    # =========================
    # Your code ends here
    # =========================


def plot_f1_curves(h0: dict, h1: dict, label0: str = "lambda=0", label1: str = "lambda>0"):
    plt.figure(figsize=(7,5))
    plt.plot(h0["val_f1"], label=f"val F1 ({label0})")
    plt.plot(h1["val_f1"], label=f"val F1 ({label1})")
    plt.xlabel("Epoch")
    plt.ylabel("Macro F1")
    plt.title("Validation F1 vs Epoch")
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
# run your code here and plot F1 curves for different lambda values
# report test F1 for both settings

hyperparams = {...}

model = HeteroCMAClassifier(...)
history_lambda_0 = train_model(model, hyperparams, lam=0.0)

model = HeteroCMAClassifier(...)
history_lambda_1 = train_model(model, hyperparams, lam=0.1) # or any lam > 0

plot_f1_curves(history_lambda_0, history_lambda_1, label0="lambda=0", label1="lambda>0")

#### 1.3.b Analysis (3 pts)
Compare λ=0 vs λ>0:
- Did alignment help convergence speed? Why?
- Did it improve peak validation F1?
- Did it over-constrain the space?


**[Answer]**

...

## Question 2 - Investigating Topology in Node-Based Classification Using GNNs (30 pts)

In this section, we will explore the impact of graph topology on node-based classification using GNNs. The experiments will focus on analyzing different topological measures, visualizing their distributions, and evaluating GCN performance on 2 graphs with different topologies.

In [ ]:
set_seed(42)


# Data Paths
G1_TRAIN_PATH = os.path.join("data", "q2_G1_train.json")
G1_EVAL_PATH  = os.path.join("data", "q2_G1_eval.json")
G2_TRAIN_PATH = os.path.join("data", "q2_G2_train.json")
G2_EVAL_PATH  = os.path.join("data", "q2_G2_eval.json")

In [ ]:
def create_Adj_matrix(N, edge_index):
    """Creates the adjacency matrix from an edge index list."""
    A = torch.zeros((N, N), dtype=torch.float)
    for idx, jdx in edge_index:
        A[idx, jdx] = 1
        A[jdx, idx] = 1
    return A

def read_json_data(file_path, has_label=True):
    """
    Loads data from a JSON file.
    Returns: list of (X, A, y) tuples.
    """
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"{file_path} not found.")

    with open(file_path, 'r') as f:
        data = json.load(f)

    # Normalize to list format even if single graph
    if not isinstance(data, list):
        data = [data]

    graph_data = []
    for item in data:
        X = torch.tensor(item['features'], dtype=torch.float).to(device)
        N = len(X)
        A = create_Adj_matrix(N, item['edge_index']).to(device)
        y = torch.tensor(item['label'], dtype=torch.long).to(device) if has_label else None
        graph_data.append((X, A, y))

    return graph_data

### Question 2.1 - Analyzing the Graphs (10 pts)

#### 2.1.a Topological and Geometric Measures (3 pts)

* Examine two topological measures (**Node Degree** and **Betweenness Centrality**) and one geometric measure (**Ollivier-Ricci Curvature**).
* Include a definition for each measure.
* Explain **mathematically** and **intuitively** what each measure quantifies and how it contributes to understanding the structure of a graph.

**[Answer]**

...

#### 2.1.b Visualizing and Comparing Topological and Geometric Measures (3 pts)

Implement the following functions to visualize and compare topological properties of two given graphs.

In [ ]:
def compute_node_degree(A):
    """Returns the degree (row sum) for each node."""
    return A.sum(dim=1)

# ==========================================
# TODO: Implement the plotting functions
# ==========================================

def plot_node_degree_distribution_two_graphs(A1, A2, label1="Graph 1", label2="Graph 2"):
    """
    Plots the node degree distributions of two graphs in a single figure.
    """
    pass

def plot_betweenness_distribution_two_graphs(A1, A2, label1="Graph 1", label2="Graph 2"):
    """
    Plots the betweenness centrality distributions of two graphs in a single figure.
    Hint: Convert adjacency matrix to NetworkX graph for calculation.
    """
    pass

def plot_ricci_curvature_distribution_two_graphs(A1, A2, label1="Graph 1", label2="Graph 2"):
    """
    Plots the Ollivier-Ricci curvature distributions for two graphs.
    Hint: Use GraphRicciCurvature to compute values, then plot densities.
    Hint: Convert adjacency matrix to NetworkX graph to use GraphRicciCurvature (see the example in graph_ricci_curvature.py).
    """
    pass

In [ ]:
# Load Data
G1_train = read_json_data(G1_TRAIN_PATH, has_label=True) 
G2_train = read_json_data(G2_TRAIN_PATH, has_label=True)

# Extract Adjacency Matrices for the first graph in each set
A1 = G1_train[0][1]
A2 = G2_train[0][1]

# Generate Plots
plot_node_degree_distribution_two_graphs(A1, A2)
plot_betweenness_distribution_two_graphs(A1, A2)
plot_ricci_curvature_distribution_two_graphs(A1, A2) # might take a bit longer to run

**[Discussion]** Compare the topologies of two graphs using the topological and geometrical measures that you computed. What are your observations?

**[Answer]**

...

#### 2.1.c Visualizing the Graphs (2 pts)

Implement the function below to generate visual representations of both graphs. Comment on the topologies.

In [ ]:
def plot_graph(A, y=None, title="Graph Visualization"):
    """
    Plots a graph defined by adjacency matrix A. 
    Nodes should be colored by labels y if provided.
    """
    # ==========================================
    # TODO: Implement graph visualization
    # ==========================================
    pass

# Visualize
plot_graph(G1_train[0][1], G1_train[0][2], title="Graph 1")
plot_graph(G2_train[0][1], G2_train[0][2], title="Graph 2")

**[Discussion]** Compare the topologies of two graphs. What are your observations?

**[Answer]**

...

#### 2.1.d Visualizing Node Feature Distributions (2 pts)

- Implement the function `plot_node_feature_dist_by_class_two_graphs` to visualize the average node feature distribution per class for two given graphs.

- Do not consider the distribution of a specific feature $ x_i $, consider the mean of the feature vector $ \mathbf{x} $ for each node.

- The function should generate a plot similar to Figure 2.



In [ ]:
def plot_node_feature_dist_by_class_two_graphs(G1, G2):
    """
    Plots the density of average node features by class for two graphs.
    G1, G2: Tuples of (X, A, y)
    """
    # ==========================================
    # TODO: Implement feature distribution plot
    # ==========================================
    pass

plot_node_feature_dist_by_class_two_graphs(G1_train[0], G2_train[0])

**[Discussion]** Analyze the results. Discuss any observed overlaps between the node feature distributions across different classes. Are the features well-separated, or do they exhibit significant overlap? Compare the distributions between the two graphs and provide insights into their differences and similariteis


**[Answer]**

...

### 2.2 -  Evaluating GCN Performance on Different Graph Structures (10 pts)


In [ ]:
# --- Helper Functions for GCN ---

def symmetric_normalize(A):
    """Performs symmetric normalization: D^{-1/2} * A * D^{-1/2}."""
    A_tilde = A + torch.eye(A.size(0)).to(A.device) # Add self-loops
    d = A_tilde.sum(dim=1)
    D_inv_sqrt = torch.diag(torch.pow(d, -0.5))
    return D_inv_sqrt @ A_tilde @ D_inv_sqrt

def prepare_dataset(dataset):
    """Pre-normalizes Adjacency matrices in a dataset list."""
    return [(X, symmetric_normalize(A), y) for X, A, y in dataset]

def plot_training_and_validation(losses, val_f1s, title="Training Progress"):
    """Plots training loss and validation F1 score."""
    epochs = range(1, len(losses) + 1)
    fig, ax1 = plt.subplots(figsize=(10, 5))

    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss', color='tab:red')
    ax1.plot(epochs, losses, color='tab:red', label='Train Loss')
    ax1.tick_params(axis='y', labelcolor='tab:red')

    ax2 = ax1.twinx()
    ax2.set_ylabel('F1 Score', color='tab:blue')
    ax2.plot(epochs, val_f1s, color='tab:blue', label='Val F1')
    ax2.tick_params(axis='y', labelcolor='tab:blue')

    plt.title(title)
    plt.show()

In [ ]:
# --- Provided Base GCN Components ---

class GCNLayer(nn.Module):
    def __init__(self, input_dim, output_dim, use_nonlinearity=True):
        super().__init__()
        self.use_nonlinearity = use_nonlinearity
        self.Omega = nn.Parameter(torch.randn(input_dim, output_dim) * (2.0 / (input_dim + output_dim))**0.5)
        self.beta = nn.Parameter(torch.zeros(output_dim))

    def forward(self, H, A_norm):
        H_agg = torch.matmul(A_norm, H)
        out = torch.matmul(H_agg, self.Omega) + self.beta
        return F.relu(out) if self.use_nonlinearity else out

# Training Loop
def train_model(model, G_train, G_eval, num_epochs=100, lr=1e-3, verbose=True, return_embeddings=False):
    X_tr, A_tr, y_tr = G_train
    X_val, A_val, y_val = G_eval
    
    optimizer = optim.AdamW(model.parameters(), lr=lr)
    loss_hist, train_f1_hist, val_f1_hist = [], [], []
    last_embeddings = None

    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        
        output = model(A_tr, X_tr, return_embeddings=return_embeddings)
        pred, embs = output if return_embeddings else (output, None)
        
        loss = F.binary_cross_entropy(pred.squeeze(), y_tr.float())
        loss.backward()
        optimizer.step()
        
        loss_hist.append(loss.item())
        
        # Evaluation
        train_f1 = evaluate_model(model, X_tr, A_tr, y_tr)
        val_f1 = evaluate_model(model, X_val, A_val, y_val)
        
        train_f1_hist.append(train_f1)
        val_f1_hist.append(val_f1)
        
        if return_embeddings: last_embeddings = embs

        if verbose and (epoch+1) % 20 == 0:
            print(f"Epoch {epoch+1:03d} | Loss: {loss.item():.4f} | Train F1: {train_f1:.3f} | Val F1: {val_f1:.3f}")

    return (loss_hist, train_f1_hist, val_f1_hist, last_embeddings) if return_embeddings else (loss_hist, train_f1_hist, val_f1_hist)

@torch.no_grad()
def evaluate_model(model, X, A, y):
    model.eval()
    pred = model(A, X)
    if isinstance(pred, tuple): pred = pred[0] # Handle return_embeddings case
    
    y_pred = (pred.squeeze() >= 0.5).long().cpu().numpy()
    y_true = y.cpu().numpy()
    return precision_recall_fscore_support(y_true, y_pred, average="micro")[2]

#### 2.2.a - Implementation of Layered GCN (1 pt)

* Modify the `GraphNeuralNetwork` class so that `num_layers` can be passed as an argument.
* Ensure it returns all embedding layers if requested (for t-SNE analysis).
* Train on G1 and G2 independently and plot results.

In [ ]:
class GraphNeuralNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers=1):
        super().__init__()
        # ==========================================
        # TODO: Define layers dynamically based on num_layers
        # ==========================================
        pass

    def forward(self, A, X, return_embeddings=False):
        # ==========================================
        # TODO: Implement forward pass storing intermediate embeddings
        # ==========================================
        pass

In [ ]:
# Prepare Data
G1_tr_p = prepare_dataset(G1_train)[0]
G1_ev_p = prepare_dataset(read_json_data(G1_EVAL_PATH))[0]
G2_tr_p = prepare_dataset(G2_train)[0]
G2_ev_p = prepare_dataset(read_json_data(G2_EVAL_PATH))[0]

MODEL_PARAMS = {"input_dim": 10, "hidden_dim": 8, "num_layers": 4}
TRAIN_PARAMS = {"num_epochs": 200, "lr": 0.0005, "verbose": True}

print("--- Training on Graph 1 ---")
set_seed(42)
model_g1 = GraphNeuralNetwork(**MODEL_PARAMS).to(device)
loss1, _, f1_1 = train_model(model_g1, G1_tr_p, G1_ev_p, **TRAIN_PARAMS)
plot_training_and_validation(loss1, f1_1, title="Graph 1 Training")

print("\n--- Training on Graph 2 ---")
set_seed(42)
model_g2 = GraphNeuralNetwork(**MODEL_PARAMS).to(device)
loss2, _, f1_2 = train_model(model_g2, G2_tr_p, G2_ev_p, **TRAIN_PARAMS)
plot_training_and_validation(loss2, f1_2, title="Graph 2 Training")

#### 2.2.b - Plotting t-SNE Embeddings (1 pt)

Use t-SNE to visualize the node embeddings from the **final hidden layer** of the GCN.

In [ ]:
def plot_tsne(embeddings, labels, title="t-SNE"):
    """
    Plots t-SNE visualization of embeddings.
    embeddings: Tensor of shape (N, D)
    labels: Tensor of shape (N,)
    """
    # ==========================================
    # TODO: Implement t-SNE plotting, you may use TSNE from sklearn (check imports)
    # ==========================================
    pass

# Retrain to capture embeddings
print("Generating Embeddings...")
set_seed(42)
_, _, _, embs1 = train_model(model_g1, G1_tr_p, G1_ev_p, num_epochs=1, return_embeddings=True, verbose=False)
plot_tsne(embs1[-2], G1_tr_p[2], title="Graph 1 Embeddings")

set_seed(42)
_, _, _, embs2 = train_model(model_g2, G2_tr_p, G2_ev_p, num_epochs=1, return_embeddings=True, verbose=False)
plot_tsne(embs2[-2], G2_tr_p[2], title="Graph 2 Embeddings")

#### 2.2.c - Training on Merged Graphs (2 pts)

* Complete `unify_two_graphs` to create $G_{union} = G_1 \cup G_2$.
* Train the GCN on this unified graph.

In [ ]:
def unify_two_graphs(G1, G2):
    """
    Merges two graphs (disjoint union).
    G1, G2: Tuples of (X, A, y)
    Returns: (X_union, A_union, y_union)
    """
    # ==========================================
    # TODO: Implement graph union
    # ==========================================
    pass

# Create Union Dataset
G_union_tr = unify_two_graphs(G1_tr_p, G2_tr_p) # Already normalized individually, but check math if re-norm needed
G_union_ev = unify_two_graphs(G1_ev_p, G2_ev_p)

# Train on Union
print("\n--- Training on Unified Graph ---")
set_seed(42)
model_union = GraphNeuralNetwork(**MODEL_PARAMS).to(device)
loss_u, _, f1_u, embs_u = train_model(model_union, G_union_tr, G_union_ev, **TRAIN_PARAMS, return_embeddings=True)

plot_training_and_validation(loss_u, f1_u, title="Unified Graph Training")
plot_tsne(embs_u[-2], G_union_tr[2], title="Unified Graph Embeddings")

#### 2.2.d Merged vs. Independent Training (6 pts)

In this part, analyze and compare the training on two graphs independently
($G_1$, $G_2$) versus training on their union $G_{\text{union}} = G_1 \cup G_2$.

Your analysis must include **both empirical evidence and conceptual reasoning**.

---

#### Required analysis

1. **Training behaviour**
   - Compare convergence trends
   - Comment on stability, speed of convergence, and any signs of underfitting or overfitting.

2. **Learned representations**
   - Compare node embeddings obtained from:
     - independent training on $G_1$ and $G_2$, and
     - merged training on $G_{\text{union}}$.

3. **Overall performance comparison**
   - Summarise differences in predictive performance and representation quality.
   - Clearly state **what changes** when moving from independent to merged training.

---

#### Hypothesis formation

- Formulate a clear hypothesis explaining **why** the observed differences occur.
- Your hypothesis should explicitly reference:
  - graph structure,
  - information mixing across graphs,
  - and the inductive bias of the model you are using.

---

#### Trade-offs and design discussion

Discuss the **advantages and drawbacks** of merged vs. independent training, considering:

- **Computational aspects**  
  (training time, memory usage, scalability)

- **Modeling aspects**  
  (representation entanglement, negative transfer, over-smoothing, or regularisation effects)

- **Practical considerations**  
  (when merged training might be desirable or risky in real-world settings)

Then propose **concrete model or optimisation or data modifications** that could mitigate the drawbacks
(e.g. architectural changes, loss terms, sampling strategies, training protocols or data modifications such as augmentations etc.).

You do **NOT** need to implement your proposal to get full points from this question. It is optional (see the bonus below).

---

#### **Bonus (optional)**

Implement your proposal for learning on  
$G_{\text{union}} = G_1 \cup G_2$ in a better way 
and compare its outcomes against both independent and naïve merged training.

Clearly explain:
- what you changed,
- why it should help,
- and whether the results support your expectation.


In [ ]:
set_seed(42)

# BONUS (optional)
# ####################################################
# MODIFY THE CODE BELOW
# ####################################################


# This part is optional. Feel free to implement a new GNN architecture, a new training procedure etc.
# You can call the provided helper functions (feel free to write your own functions for plotting)
# plot_training_and_validation
# plot_tsne

# ####################################################
# END OF MODIFICATION
# ####################################################

### 2.3 Topological Changes to Improve Training (10 points)


#### 2.3.a - Plot the Ricci Curvature for each edge (1 pt)

- Create a bar plot showing the Ricci curvature for each edge. The x-axis should represent the edges, and the y-axis should represent their corresponding Ricci curvature values.

In [ ]:
def plot_ricci_per_edge(A):
    """
    Plots the Ollivier-Ricci curvature for each edge
    defined by adjacency matrix A (PyTorch tensor).

    Parameters
    ----------
    A : torch.Tensor
        Adjacency matrix for an undirected graph (NxN).
    
    Follow the example in graph_ricci_curvature python script.

    You can use nx.from_numpy_array to convert the adjacency matrix to a NetworkX graph.
    
    """
	# ==========================================
    # TODO: Implement Ricci edge plot
    # ==========================================
    pass

set_seed(42)

plot_ricci_per_edge(G1_train[0][1]) # might take some time to run
plot_ricci_per_edge(G2_train[0][1]) # might take some time to run

#### 2.3.b Extreme Topological Scenarios (4 points)

In this question, you will explore limit cases of graph topology to understand what role the graph
structure plays in GNN learning. Please do the following tasks for both graphs (G1 and G2). If your reasoning and technique is the same for both graphs, you do not need to explain the same thing twice. Still, please run your implementation on both graphs. 

1. **Topology removal**  
   Construct a version of the graph where neighborhood aggregation becomes meaningless, such that
   the GNN behaves similarly to a node-feature-only MLP.
   - Describe precisely how you modified the graph structure.
   - Train the GCN on this modified graph and report the performance.

2. **Oracle topology**  
   Construct an idealized graph assuming label information is available **only for analysis purposes**
   (not as input features).
   - This means you are allowed to look at the labels to design or modify the graph, but you are not allowed to give the labels to the model as input features. The model still receives only node features and graph structure and is trained normally using labels only in the loss function. 
   - Describe what edges you add or remove and why this topology should be easy for a GNN.
   - Train the GCN on this graph and report the performance.

3. **Analysis:**  
   Compare the results from:
   - the original graph,
   - the topology-removed graph,
   - the oracle graph.
   
   Discuss:
   - What these extreme cases reveal about the relationship between graph topology and node labels.
   - Why the original GCN behaves the way it does on this dataset.


**Topology removal**

Explain your reasoning and methodology

**[Answer]**

...

In [ ]:
def topology_removal(G):
    """Removes informative topology."""
    X, A, y = G
    # ==========================================
    # TODO: Modify A
    # ==========================================
    return X, A, y

**Oracle topology**

Explain your reasoning and methodology

**[Answer]**

...

In [ ]:
def oracle_topology(G):
    """Constructs ideal topology. You may use the labels y to modify A."""
    X, A, y = G
    # ==========================================
    # TODO: Modify A
    # ==========================================
    return X, A, y

In [ ]:
def run_topology_experiment(strategy_fn, G_tr, G_ev, title):
    print(f"\n--- Experiment: {title} ---")
    
    # Apply strategy and re-normalize
    G_tr_mod = prepare_dataset([strategy_fn(G_tr)])[0]
    G_ev_mod = prepare_dataset([strategy_fn(G_ev)])[0]
    
    set_seed(42)
    model = GraphNeuralNetwork(**MODEL_PARAMS).to(device)
    loss, _, f1 = train_model(model, G_tr_mod, G_ev_mod, **TRAIN_PARAMS)
    print(f"Final Val F1: {f1[-1]:.4f}")
    plot_training_and_validation(loss, f1, title=title)

# Run Experiments
# for both G1 and G2
set_seed(42)

run_topology_experiment(topology_removal, G1_train[0], read_json_data(G1_EVAL_PATH)[0], "Topology Removal (G1)")
run_topology_experiment(oracle_topology, G1_train[0], read_json_data(G1_EVAL_PATH)[0], "Oracle Topology (G1)")

run_topology_experiment(topology_removal, G2_train[0], read_json_data(G2_EVAL_PATH)[0], "Topology Removal (G2)")
run_topology_experiment(oracle_topology, G2_train[0], read_json_data(G2_EVAL_PATH)[0], "Oracle Topology (G2)")

#### 2.3.c Graph Augmentation Without Labels (5 pts)

In Question 2.3.b, you constructed an **Oracle graph** using labels in order to estimate the
*theoretical upper bound* on performance.  
That experiment was **diagnostic only**.

In this question, your goal is to design a **practical graph augmentation strategy**
that could be applied **without access to labels**.

---

#### Task

Implement a graph augmentation method that modifies the adjacency matrix $A$
using **only**:
- node features $X$, and/or
- the original graph topology $A$ itself.


---

#### Important considerations

- **No labels**  
  You are **strictly forbidden** from using the labels $y$ (directly or indirectly)
  to decide on modification of graph.

- **Generalisability**  
  Your method should ideally apply to **both graphs** $G_1$ and $G_2$.
  If your approach only works well for one graph, you may implement
  two separate versions of `augment_graph`.
  In that case, clearly explain:
  - why the method does not generalise,
  - what structural differences between the graphs cause this limitation.

- **Performance is not the only objective**  
  It is expected that your augmentation improves classification performance
  relative to training on the original $G_1$ and $G_2$.
  If performance does not improve, you may instead argue that your method:
  - reduces computational cost,
  - improves training stability,
  - simplifies the graph structure,
  - etc.

  In all cases, you must clearly explain **what improves (or fails to improve) and why**.

---

#### Evaluation and analysis

- Compare results obtained from:
  - the original graphs,
  - your augmented graphs,
- Use consistent models, training settings, hyperparameters to be fair.

---




In [ ]:
def augment_graph(G):
    """
    Augments the graph structure using only features (X) and existing topology (A).
    
    Args:
        G: Tuple (X, A, y)
    
    Returns:
        X, A_new, y
        
    IMPORTANT: You must NOT use 'y' (labels) to construct A_new. 
    """
    X, A, y = G
    
    # ==========================================
    # TODO: Implement your augmentation strategy
    # ==========================================
    
    return X, A, y

set_seed(42)

print("\n--- Realistic Augmentation Training ---")
# We use the same runner as before, but this time using the "augment_graph" function
run_topology_experiment(augment_graph, G1_train[0], read_json_data(G1_EVAL_PATH)[0], "Realistic Augmentation (G1)")

run_topology_experiment(augment_graph, G2_train[0], read_json_data(G2_EVAL_PATH)[0], "Realistic Augmentation (G2)")

#### Interpretation

Explain:
- your methodology,
- what change you introduced to the graph,
- what assumption this change encodes about the data,
- and why this assumption helps or fails on this dataset.

Support your discussion with **quantitative results and plots** where appropriate.

**[Answer]**

...

## Question 3 - Mystery Graph Investigation (30 pts)

In this question, you will work with a **mystery graph dataset** and investigate the behavior of graph-based learning.

You will proceed in three stages:

1. **Train baseline models** and observe their performance.
2. **Diagnose and explain** unexpected behavior using analysis and visualisation.
3. **Design and implement your own method** to improve performance.

You are not told anything about the structure of the dataset in advance.
Your task is to *discover* what is happening and respond accordingly.

### Rules and Constraints

- You may only use **NumPy** and **PyTorch**.
- Do not modify the dataset.
- You may define additional helper functions if needed.
- Written answers must be provided in the Markdown cells.

This question rewards **reasoning and diagnosis**, not just performance.

---


In [ ]:
set_seed(42)

In [ ]:
with open("data/mystery_graph.json", "r") as f:
    data = json.load(f)

x = torch.tensor(data["x"], dtype=torch.float32, device=device)
y = torch.tensor(data["y"], dtype=torch.long, device=device)

src = torch.tensor(data["src"], dtype=torch.long, device=device)
dst = torch.tensor(data["dst"], dtype=torch.long, device=device)
edge_w = torch.tensor(data["edge_w"], dtype=torch.float32, device=device)

train_mask = torch.tensor(data["train_mask"], dtype=torch.bool, device=device)
val_mask   = torch.tensor(data["val_mask"], dtype=torch.bool, device=device)
test_mask  = torch.tensor(data["test_mask"], dtype=torch.bool, device=device)

N, D = x.shape
C = int(data["num_classes"])

print(f"N={N}, D={D}, C={C}")
print(f"Directed edges: {src.numel()}")


### 3.1 Baseline Models

You will first train two baseline models:

- **MLP**: ignores the graph completely.
- **GCN**: uses mean aggregation over neighbors.

At this stage, **do not try to improve anything**.
Simply train both models and observe their performance.

---


In [ ]:
def macro_f1(pred, target, num_classes):
    f1s = []
    for c in range(num_classes):
        tp = ((pred == c) & (target == c)).sum().item()
        fp = ((pred == c) & (target != c)).sum().item()
        fn = ((pred != c) & (target == c)).sum().item()
        prec = tp / (tp + fp + 1e-12)
        rec  = tp / (tp + fn + 1e-12)
        f1 = 0.0 if (prec + rec) == 0 else 2 * prec * rec / (prec + rec + 1e-12)
        f1s.append(f1)
    return float(sum(f1s) / len(f1s))


In [ ]:

def mean_aggregate(h, src, dst, num_nodes):
    out = torch.zeros((num_nodes, h.size(1)), device=h.device)
    out.index_add_(0, dst, h[src])
    deg = torch.zeros((num_nodes,), device=h.device)
    deg.index_add_(0, dst, torch.ones_like(dst, dtype=h.dtype))
    return out / (deg.unsqueeze(-1) + 1e-12)


class MLP(nn.Module):
    def __init__(self, d_in, d_hidden, n_classes, dropout=0.2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d_in, d_hidden),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_hidden, n_classes),
        )
    def forward(self, x):
        return self.net(x)


class GCN(nn.Module):
    def __init__(self, d_in, d_hidden, n_classes, dropout=0.2):
        super().__init__()
        self.lin1 = nn.Linear(d_in, d_hidden)
        self.lin2 = nn.Linear(d_hidden, n_classes)
        self.dropout = dropout

    def forward(self, x, src, dst, num_nodes):
        h = mean_aggregate(x, src, dst, num_nodes)
        h = F.relu(self.lin1(h))
        h = F.dropout(h, p=self.dropout, training=self.training)
        h = mean_aggregate(h, src, dst, num_nodes)
        return self.lin2(h)


In [ ]:
def train_model(model, forward_fn, epochs=80, lr=1e-3):
    opt = torch.optim.Adam(model.parameters(), lr=lr)

    best_val = -1.0
    best_state = None
    history = {"train_f1": [], "val_f1": []}

    for ep in range(1, epochs + 1):
        model.train()
        logits = forward_fn()
        loss = F.cross_entropy(logits[train_mask], y[train_mask])

        opt.zero_grad()
        loss.backward()
        opt.step()

        model.eval()
        with torch.no_grad():
            pred = forward_fn().argmax(dim=-1)
            tr = macro_f1(pred[train_mask], y[train_mask], C)
            va = macro_f1(pred[val_mask], y[val_mask], C)

        history["train_f1"].append(tr)
        history["val_f1"].append(va)

        if va > best_val:
            best_val = va
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}

        if ep == 1 or ep % 10 == 0:
            print(f"Epoch {ep:03d} | loss={loss.item():.4f} | trainF1={tr:.3f} | valF1={va:.3f}")

    if best_state is not None:
        model.load_state_dict(best_state, strict=True)

    return history


@torch.no_grad()
def eval_model(model, forward_fn):
    model.eval()
    logits = forward_fn()
    pred = logits.argmax(dim=-1)
    return (
        macro_f1(pred[train_mask], y[train_mask], C),
        macro_f1(pred[val_mask], y[val_mask], C),
        macro_f1(pred[test_mask], y[test_mask], C),
    )


In [ ]:
set_seed(42)

mlp = MLP(D, 64, C).to(device)
hist_mlp = train_model(mlp, lambda: mlp(x))
print("MLP  train/val/test:", eval_model(mlp, lambda: mlp(x)))

set_seed(42)

gcn = GCN(D, 64, C).to(device)
hist_gcn = train_model(gcn, lambda: gcn(x, src, dst, N))
print("GCN  train/val/test:", eval_model(gcn, lambda: gcn(x, src, dst, N)))

plt.figure(figsize=(7,4))
plt.plot(hist_mlp["val_f1"], label="MLP")
plt.plot(hist_gcn["val_f1"], label="GCN")
plt.xlabel("Epoch")
plt.ylabel("Validation Macro-F1")
plt.legend()
plt.tight_layout()
plt.show()


### 3.2 Explaining the Results (15 pts)

You should observe that **the MLP performs better than the GCN**.

This is not what one might expect if the graph were always helpful.

#### Task
Using **quantitative measures and visualisations**, explain:

1. Why does the GCN perform worse than a feature-only MLP? Considering a GCN does neighborhood aggregation, it is not expected an MLP to perform better than a GCN. Explain this phenomenon. 
2. What is the relationship between node labels and the graph structure? 
3. What kind of information does **mean aggregation** preserve or destroy here?
4. How is this phenomenon related to homophily in graphs and GNNs?

You can reuse or extend analyses from earlier or your are encouraged to introduce new diagnostics if needed.

---


In [ ]:
# TODO
# Add diagnostics to support your explanation.
# Make an analysis on graph and model properties to explain the results you obtained.
# What properties of the graph and the model do you think contributed to the results you obtained? 
# Write your code that supports your explainations to these questions.

pass


#### 3.2 Answer

**Why does the GCN perform worse than a feature-only MLP? Considering a GCN does neighborhood aggregation, it is not expected an MLP to perform better than a GCN. Explain this phenomenon.**  
...

**What is the relationship between node labels and the graph structure?**  
...

**What kind of information does \textbf{mean aggregation} preserve or destroy here?**  
...

**How is this phenomenon related to homophily in graphs and GNNs?**
...

---


### 3.3 Designing a Better Graph Method (15 pts)

The GCN fails because of *how* it aggregates information from neighbors.

Your task is to design a **new message passing method** that:

- uses the graph,
- but avoids the failure mode of mean aggregation on this dataset.

Your solution needs to

- address the failure reason that you discovered in Stage 2,
- be sophisticated rather than simply adding more layers and increasing number of learnable parameters.

If your reasoning is about the optimization strategy (e.g., new optimizers instead of Adam) rather than the architecture itself, you need to apply your new optimization strategy to classical GCN and MLP too in order to be fair.

There is **no single correct architecture**.
What matters is that your design is **well-motivated** and **improves performance**.

---


In [ ]:
# TODO:
# Define any helper function(s) you need.
# Feel free to modify the following class according to your needs, it is given just as a skeleton code.

pass


class MyGNN(nn.Module):
    """
    Your custom graph model.
    """
    def __init__(self, d_in, d_hidden, n_classes):
        super().__init__()
        # TODO: define layers
        pass

    def forward(self, x, src, dst, edge_w, num_nodes):
        # TODO: implement forward pass
        pass


In [ ]:
set_seed(42) # use the same seed for reproducibility

# TODO:
# 1) Instantiate your model
# 2) Train it using train_model
# 3) Report train/val/test F1
# 4) Plot validation curve alongside MLP and GCN

pass


#### 3.3 Final Explanation

Describe:

1. What failure mode you identified in the GCN.
2. How your method addresses this failure.
3. Why your method improves performance on this dataset.
4. What assumptions your method makes about the graph.

Your answer should clearly connect:
- dataset structure,
- aggregation behavior,
- and model design.

---


**[Answer]**

...